In [1]:
import pandas as pd

import tweepy
import praw
import config

from bots.twitter import TwitterBot

2023-03-17 16:36:59,127 - INFO     | config     | Loading environment variables
2023-03-17 16:36:59,128 - WARNING  | config     | Directory data already exists


In [2]:
twitter_bot = TwitterBot(bearer=config.TWTR_BEARER_TOKEN,
                         api=config.TWTR_API,
                         api_secret=config.TWTR_API_SECRET,
                         access=config.TWTR_ACCESS_TOKEN,
                         access_secret=config.TWTR_ACCESS_TOKEN_SECRET)

In [3]:
query = 'entity:"Nike" -is:retweet lang:en'

In [4]:
tweets = twitter_bot.get_recent_tweets(query=query, limit=500)

In [5]:
df = pd.DataFrame.from_dict(tweets)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          500 non-null    int64              
 1   user        500 non-null    int64              
 2   text        500 non-null    object             
 3   created_at  500 non-null    datetime64[ns, UTC]
 4   entities    500 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(2), object(2)
memory usage: 19.7+ KB


In [8]:
df.to_parquet(f'{config.DATA_DIR}/test.parquet', index=False)